In [25]:
from torch.cuda.amp import GradScaler, autocast

In [1]:
import torch
import numpy as np
import pandas as pd


# read whole corpus
processed_df = pd.read_csv('/cluster/scratch/goezsoy/nlp_lss_datasets/processed_df.csv')

# work on subset of speakers
temp_bioguides = ['C001074']
temp_df = processed_df[processed_df.bioguide_id.isin(temp_bioguides)]


/cluster/scratch/goezsoy/miniconda3/envs/cond_text_gen_project/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
target_text= "Majority Leader Reid"
processed_df[processed_df.speech.apply(lambda row: True if target_text in row else False) == True].speech.iloc[:6].values

,Unnamed: 0,bioguide_id,speech,first_name,last_name,birthday,gender,term_type,term_start,term_end,term_state,term_party,token_count
4,4,A000014,". Mr. Speaker, in support of our substitute am...",Neil,Abercrombie,1938-06-26,M,H,2009-01-06,2010-03-01,HI,Democrat,456
13,13,A000014,". Mr. Speaker, I would like to enter into the ...",Neil,Abercrombie,1938-06-26,M,H,2009-01-06,2010-03-01,HI,Democrat,1710
19,20,A000014,". Madam Speaker, mahalo nui. Thank you very mu...",Neil,Abercrombie,1938-06-26,M,H,2009-01-06,2010-03-01,HI,Democrat,754
34,35,A000022,". Mr. Speaker, I am pleased to yield 2 minutes...",Gary,Ackerman,1942-11-19,M,H,2009-01-06,2011-01-03,NY,Democrat,202
41,42,A000022,". Madam Speaker, I strongly support the resolu...",Gary,Ackerman,1942-11-19,M,H,2009-01-06,2011-01-03,NY,Democrat,535
...,...,...,...,...,...,...,...,...,...,...,...,...,...
206046,208129,Z000017,". Mr. Speaker, I yield myself such time as I m...",Lee,Zeldin,1980-01-30,M,H,2015-01-06,2017-01-03,NY,Republican,1323
206048,208131,Z000017,". Mr. Speaker, I thank the gentleman from Tenn...",Lee,Zeldin,1980-01-30,M,H,2015-01-06,2017-01-03,NY,Republican,2638
206053,208136,Z000017,". Thank you, Mr. DeSantis. Thank you for your ...",Lee,Zeldin,1980-01-30,M,H,2015-01-06,2017-01-03,NY,Republican,708
206059,208142,Z000017,". Mr. Speaker, I oppose U.N. Security Council ...",Lee,Zeldin,1980-01-30,M,H,2015-01-06,2017-01-03,NY,Republican,267


In [ ]:
# loading model for generation
from model import Net

# setting device on GPU if available, else CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

model = Net().to(device)

checkpoint = torch.load('../checkpoints/speaker_name_as_prompt_deneme_best.pt',map_location=device)

model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

In [ ]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-125M")
tokenizer.pad_token = tokenizer.eos_token

inputs = tokenizer('Travis Chil', return_tensors="pt")

gen_tokens = model.gpt_neo.generate(
    inputs.input_ids,
    do_sample=True,
    num_beams=1,
    temperature=0.5,
    max_length=100,
)
tokenizer.batch_decode(gen_tokens)[0]